In [1]:
# required Python imports
import numpy as np
from types import SimpleNamespace

import ftir_funct as f
np.set_printoptions(suppress=True)

module FTIR v.2024.3.18 imported


## Generate database

In [2]:
f.explore_Euler_space(step=7, lower_bounds=(0, 0, 0), upper_bounds=(89, 89, 179))

array([[  0,   0,   0],
       [  0,   0,   7],
       [  0,   0,  14],
       ...,
       [ 84,  84, 161],
       [ 84,  84, 168],
       [ 84,  84, 175]])

In [3]:
# generate Euler angles and store in the database
database = SimpleNamespace(euler=f.explore_Euler_space(step=7, lower_bounds=(0, 0, 0), upper_bounds=(89, 89, 179)))
database.euler.shape

(4394, 3)

In [5]:
# Generate a mesh of values defining the reference transmissión envelope
polar, azimuths = f.regular_S2_grid(n_squared=500)
T = f.Tvalues(trans=(90, 50, 20), azimuth=azimuths, polar=polar)
x, y, z = f.sph2cart(T, azimuths, polar)

# Generate 16 measures at different azimuth angles
angles = np.arange(0, 360, 45/2)

# initialize variables
T_vals = []
azi_vals = []

for euler in database.euler:
    # rotate
    x2, y2, z2 = f.rotate(coordinates=(x, y, z), euler_ang=euler)

    # extract XY intersection
    xy_vectors = f.extract_XY_section_fast2(x2, y2, z2)

    # get the indexes of specific angles
    indexes = f.find_nearest(xy_vectors['angles'], angles)

    # append values
    T_vals.append(xy_vectors.loc[indexes, ['T']].T.values.tolist()[0])
    azi_vals.append(xy_vectors.loc[indexes, ['angles']].T.values.tolist()[0])

# store values in database
database.T_values = np.array(T_vals)
database.azimuths = np.array(azi_vals)

In [6]:
print(database.euler.shape)
print(database.T_values.shape)
print(database.azimuths.shape)

(4394, 3)
(4394, 16)
(4394, 16)


In [7]:
print('Euler angles: ', database.euler[-1])
print('T values: ', np.around(database.T_values[-1], 1))
print('Azimuths: ', np.around(database.azimuths[-1], 1))

Euler angles:  [ 84  84 175]
T values:  [50.3 44.5 33.4 23.7 20.9 26.6 37.7 47.5 50.3 44.5 33.4 23.7 20.9 26.6
 37.7 47.5]
Azimuths:  [  0.2  22.5  45.2  67.4  90.  112.3 135.  157.5 180.  202.5 225.2 247.5
 270.  292.3 315.  337.5]


In [10]:
measures = np.column_stack((database.T_values[-1],
                            database.azimuths[-1],
                            np.full_like(database.azimuths[-1], 90)))

estimate_1 = f.find_orientation(measurements=measures, params=(90, 50, 20), silent=False)
estimate_1.x

Calculated Orientation: [ 84.  84. 175.]


array([ 83.99720167,  83.99909925, 174.99923652])

## Estimate
TODO: use a proper params in the target spectrum!

In [ ]:
# 
default_algorithm = np.empty(database.euler.shape[0])
diffEvo_algorithm = np.empty(database.euler.shape[0])
duaAnne_algorithm = np.empty(database.euler.shape[0])

for index, orientation in enumerate(database.euler):
    # print('Real:', np.around(orientation, 0))

    measures = np.column_stack((database.T_values[index],
                                database.azimuths[index],
                                np.full_like(database.azimuths[index], 90)))

    estimate_1 = f.find_orientation(measurements=measures, params=(90, 50, 20))
    default_algorithm[index] = f.calc_misorientation(orientation, estimate_1.x)

    estimate_2 = f.find_orientation_diffevol(measurements=measures, params=(90, 50, 20))
    diffEvo_algorithm[index] = f.calc_misorientation(orientation, estimate_2.x)

    estimate_3 = f.find_orientation_annealing(measurements=measures, params=(90, 50, 20))
    duaAnne_algorithm[index] = f.calc_misorientation(orientation, estimate_3.x)


## Statistical assessment